## 2.2 - Chain of Thought Prompting

**Chain of Thought** (CoT) is a concept in natural language processing (NLP) that enables 
language models to reason through tasks in a structured, step-by-step manner, similar to human 
thinking. Rather than generating isolated responses, a model using Chain of Thought builds on 
prior information, creating responses that follow a logical flow and ensuring a coherent, 
context-aware dialogue. This approach is particularly valuable in complex or multi-step queries,
where understanding and maintaining context over multiple interactions is essential.

## Why Chain of Thought Matters

In traditional LLM models, responses can sometimes feel disjointed or context-free, especially with complex questions. Chain of Thought addresses these challenges by adding a reasoning layer, allowing the model to generate more accurate and relevant responses. This logical progression offers key benefits:

1. **Coherence**: Responses follow a natural flow, making it easier for users to follow and engage with the conversation.
2. **Context Retention**: By building on previous information, the model maintains context across multiple steps, crucial for handling complex questions.
3. **Increased Accuracy**: Logical progression reduces the likelihood of irrelevant or incorrect information, enhancing the reliability of responses.


## Preparation

In [1]:
#!pip install -q langchain==0.1.9 openai==1.13.3
!pip install -q langchain-openai langchain
#!pip uninstall langchain openai


[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
#from langchain_community.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

In [3]:
INFERENCE_SERVER_URL = os.getenv('API_URL_GRANITE')
MODEL_NAME = "granite30-8b"
API_KEY= os.getenv('API_KEY')

### Create the LLM instance

In [4]:
# LLM definition
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base= f"{INFERENCE_SERVER_URL}/v1",
    model_name=MODEL_NAME,
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

### Create the Prompt

In [5]:
template = """\
<|start_of_role|>system<|end_of_role|>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect. 
If you don't know the answer to a question, please don't share false information.
<|end_of_text|>

<|start_of_role|>user<|end_of_role|>
Human: {input}
<|end_of_text|>

<|start_of_role|>assistant<|end_of_role|>
"""

PROMPT = PromptTemplate(input_variables=["input"], template=template)

In [6]:
conversation = LLMChain(llm=llm,
                            prompt=PROMPT_LLM,
                            verbose=False,
                            )

/tmp/ipykernel_1129/2670321090.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(llm=llm,


### Let's ask a simple question **WITHOUT** Chain of Thought in action

In [7]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation.predict(input=first_input);

 To design a financial report based on reports from larger Financial Services Institutions (FSIs), you should focus on key financial metrics such as revenue, net income, assets, liabilities, and equity. Additionally, consider analyzing their risk management strategies, investment portfolios, and operational efficiency. It's also beneficial to compare your institution's performance against the benchmarks set by these larger FSIs to identify areas for improvement and opportunities for growth. Furthermore, include an executive summary that provides an overview of the findings and any relevant conclusions or recommendations.

### LLM with Chain of Thought in the Prompt

In [8]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful, and honest assistant. Always be as helpful as possible while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
When answering, think step-by-step and explain your reasoning before providing the final answer.
Explain it with bullet points if it's needed but using reasoning and step-by-step.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something incorrect.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT_COT = PromptTemplate(input_variables=["input"], template=template)

### Chain of Thought Prompt

Let's ask the same question with Chain of Thought

* When answering, think step-by-step and explain your reasoning before providing the final answer.
* Explain it with bullet points if it's needed but using reasoning and step-by-step.

In [9]:
conversation_cot = LLMChain(llm=llm,
                            prompt=PROMPT_COT,
                            verbose=False,
                            )

In [10]:
first_input = "I want to design a finantial report around the report from the larger FSIs. What I should focus?"
conversation_cot.predict(input=first_input);

 To design a financial report based on the reports from larger Financial Services Institutions (FSIs), here are some key areas you should focus on:

1. **Financial Statements**: Analyze the income statements, balance sheets, and cash flow statements of the FSIs. These statements provide a comprehensive overview of their financial performance, including revenue, expenses, assets, liabilities, and cash flow.

2. **Key Performance Indicators (KPIs)**: Identify the KPIs that are commonly used by the FSIs and compare them with your own organization's performance. Some common KPIs include Return on Equity (ROE), Return on Assets (ROA), Net Interest Margin, and Efficiency Ratio.

3. **Risk Management**: Examine the risk management practices of the FSIs. This includes their approach to credit risk, market risk, operational risk, and liquidity risk. Understanding these practices can help you identify potential risks in your own organization and develop strategies to mitigate them.

4. **Regulat

## Enhancing LLMs Performance with Chain of Thought

By simulating a thought process, Chain of Thought enhances NLP models by:

- **Improving Context Understanding**: The model remembers previous interactions, generating responses that are relevant and contextually aware.
- **Boosting User Experience**: Users benefit from a more natural and satisfying interaction, as the model “thinks” through its responses.
- **Increasing Flexibility**: CoT enables models to handle multi-part questions, making them versatile for applications like virtual assistants, customer service bots, and educational tools.

Incorporating Chain of Thought into LLMs frameworks, such as LangChain, enables language models to tackle complex tasks, making them more effective for real-world applications where logical reasoning and clarity are paramount.